<a href="https://colab.research.google.com/github/abhijithneilabraham/2000Line/blob/master/embeddingtryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install bert_embedding
from bert_embedding import BertEmbedding
from datetime import datetime
import concurrent.futures
import pandas as pd

start_ts = datetime.now()
bert_embedding=BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_uncased')
loaded_ts = datetime.now()
load_time = loaded_ts - start_ts

phrases = [['checking account'], ['savings account'], ['account balance']]

h=pd.read_csv('test1.csv')
x1=[[str(i)] for i in h['identifier']]
def get_embedding(text):
    res = bert_embedding(text, 'sum')
    return res

with concurrent.futures.ProcessPoolExecutor() as executor:
    embeddings = executor.map(get_embedding, x1)
    
compute_ts1 = datetime.now()
compute_time1 = compute_ts1 - loaded_ts

with concurrent.futures.ProcessPoolExecutor() as executor:
    embeddings = executor.map(get_embedding, x1)
compute_ts2 = datetime.now()
compute_time2 = compute_ts2 - compute_ts1

for embedding in embeddings:
    print(embedding)
print(load_time.total_seconds(), compute_time1.total_seconds(), compute_time2.total_seconds())


In [3]:
!pip install bert_embedding
from bert_embedding import BertEmbedding
import pandas as pd


bert_embedding=BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased')

phrases = [['checking account'], ['savings account'], ['account balance']]
h=pd.read_csv('test1.csv')
x1=[[str(i)] for i in h['identifier']]

def get_embedding(text):
    res = bert_embedding(text, 'avg')
    return res

import multiprocessing as mp
pool = mp.Pool(mp.cpu_count()-1)
results = pool.map(get_embedding, [row for row in x1])
pool.close()
print(results)

     |████████████████████████████████| 29.6MB 1.6MB/s 
     |████████████████████████████████| 215kB 40.8MB/s 
     |████████████████████████████████| 13.8MB 34.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ff/48/ac/a77c79aa416ba6dd7bf487f2280b0471034f66141617965914
Successfully built gluonnlp
ERROR: spacy 2.1.6 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: fastai 1.0.55 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.4
    Uninstalling

Vocab file is not found. Downloading.
[[(['isprime'], [array([-0.49705517,  0.16107166,  0.13566259, ...,  0.39847922,
       -0.0946147 ,  0.38457203], dtype=float32)])], [(['MAX'], [array([-0.582132  , -0.5979823 , -0.2819156 , ..., -0.37734562,
        0.02318024, -0.46172535], dtype=float32)])], [(['i'], [array([-0.2651239 , -0.8707455 , -0.05090838, ..., -0.39729232,
       -0.07990786, -0.37390482], dtype=float32)])], [(['j'], [array([-0.25441223, -1.0615157 , -0.13851798, ..., -0.5649176 ,
       -0.04482983, -0.26304406], dtype=float32)])], [(['i'], [array([-0.2651239 , -0.8707455 , -0.05090838, ..., -0.39729232,
       -0.07990786, -0.37390482], dtype=float32)])], [(['i'], [array([-0.2651239 , -0.8707455 , -0.05090838, ..., -0.39729232,
       -0.07990786, -0.37390482], dtype=float32)])], [(['i'], [array([-0.2651239 , -0.8707455 , -0.05090838, ..., -0.39729232,
       -0.07990786, -0.37390482], dtype=float32)])], [(['N'], [array([-0.63991535, -0.1393497 ,  0.4788293 , ..., -0.

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: G

In [0]:
 cat /proc/cpuinfo

In [3]:
import multiprocessing
print(multiprocessing.cpu_count())

2
